In [1]:
# IMPORT AND DISPLAY SETTINGS


import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
# import xgboost as xgb
# import lightgbm as lgb
import pandas as pd

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format



In [2]:
user_data = pd.read_csv(r"processed-data\loan-processed-data.csv")

In [3]:
user_data['Default'] = user_data['Default'].astype(int)
user_data.head()

,Client_Income_Type_Govt Job,Client_Income_Type_Others,Client_Income_Type_Retired,Client_Income_Type_Service,Client_Income_Type_Unemployed,Client_Education_Graduation dropout,Client_Education_Junior secondary,Client_Education_Post Grad,Client_Education_Secondary,Client_Marital_Status_M,Client_Marital_Status_S,Client_Marital_Status_W,Client_Gender_Male,Loan_Contract_Type_RL,Client_Housing_Type_Home,Client_Housing_Type_Municipal,Client_Housing_Type_Office,Client_Housing_Type_Rental,Client_Housing_Type_Shared,Client_Permanent_Match_Tag_Yes,Client_Contact_Work_Tag_Yes,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Accompany_Client,Population_Region_Relative,Age_Days,Employed_Days,Registration_Days,ID_Days,Mobile_Tag,Homephone_Tag,Workphone_Working,Client_Family_Members,Cleint_City_Rating,Application_Process_Day,Application_Process_Hour,Score_Source_2,Score_Source_3,Phone_Change,Credit_Bureau,Default,Occupation_Risk_Level,Org_Type_Binned
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,1.00,12142509.00,6750.00,0.00,0.00,1.00,0.00,0.00,61190.55,3416.85,0.00,0.03,13957.00,1062.00,6123.00,383.00,1.00,0.00,0.00,2.00,2.00,6.00,17.00,0.48,0.55,63.00,1.00,0,2.00,3.00
1,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,1.00,12138936.00,20250.00,1.00,0.00,1.00,1.00,0.00,15282.00,1826.55,0.00,0.01,14162.00,4129.00,7833.00,21.00,1.00,0.00,1.00,2.00,2.00,3.00,10.00,0.22,0.55,755.00,1.00,0,2.00,2.00
2,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,12181264.00,18000.00,0.00,0.00,1.00,0.00,1.00,59527.35,2788.20,0.00,0.02,16790.00,5102.00,4493.00,331.00,1.00,0.00,0.00,2.00,2.00,4.00,12.00,0.55,0.33,277.00,0.00,0,2.00,3.00
3,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,1.00,12188929.00,15750.00,0.00,0.00,1.00,1.00,0.00,53870.40,2295.45,0.00,0.01,23195.00,12019.50,4493.00,775.00,1.00,0.00,0.00,2.00,2.00,2.00,15.00,0.14,0.63,1700.00,3.00,0,2.00,1.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,1.00,12133385.00,33750.00,1.00,0.00,1.00,0.00,2.00,133988.40,3547.35,0.00,0.02,11366.00,2977.00,5516.00,4043.00,1.00,0.00,0.00,4.00,2.00,3.00,12.00,0.30,0.36,674.00,1.00,0,2.00,3.00


In [4]:
#  We do not need to scale the following:
# These are already encoded as binary (0/1), ordinal, or categorical numericals:

# 'Client_Income_Type_Govt Job', 'Client_Income_Type_Service', etc. → One-hot encoded → No scaling needed

# 'Client_Education_Secondary', 'Client_Marital_Status_S', etc. → One-hot encoded →  No scaling

# 'Client_Gender_Male', 'Loan_Contract_Type_RL', etc. →  No scaling

# 'Car_Owned', 'Bike_Owned', 'Active_Loan', 'House_Own', 'Mobile_Tag', 'Workphone_Working', etc. →  No scaling

# 'Occupation_Risk_Level', 'Org_Type_Binned' → Ordinal (already numeric mapping) →  Don’t scale unless their range dominates

#  We should scale the following numerical columns because they:
# Span large ranges (e.g. income, age, credit amount)

# Will skew distance-based models (like SVM, KNN, Logistic Regression)

columns_to_scale = [
        'Client_Income', 'Credit_Amount', 'Loan_Annuity',
        'Population_Region_Relative', 'Age_Days', 'Employed_Days',
        'Registration_Days', 'ID_Days', 'Score_Source_2',
        'Score_Source_3', 'Phone_Change', 'Credit_Bureau',
        'Client_Family_Members'
    ]

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import pandas as pd

def stratified_split_and_scale(df, target_col, test_size=0.2, random_state=42):
    # Split features and target
    X = df.drop(columns=[target_col])
    y = df[target_col]

    # Stratified train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=random_state
    )

    # Identify numeric columns
    numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns

    # Initialize the scaler
    scaler = StandardScaler()
    # scaler = MinMaxScaler()


    # Fit on training data and transform both
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()

    X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
    X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

    return X_train_scaled, X_test_scaled, y_train, y_test, scaler

X_train, X_test, y_train, y_test, scaler = stratified_split_and_scale(user_data, target_col='Default')
# Check distribution
print("Train target distribution:")
print(y_train.value_counts(normalize=True))

print("\nTest target distribution:")
print(y_test.value_counts(normalize=True))


# Logistic Regression with class weight balancing
model = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    random_state=28
)



# Fit the model
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Train target distribution:
Default
0   0.92
1   0.08
Name: proportion, dtype: float64

Test target distribution:
Default
0   0.92
1   0.08
Name: proportion, dtype: float64
Confusion Matrix:
 [[15926  6477]
 [  622  1347]]

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.71      0.82     22403
           1       0.17      0.68      0.28      1969

    accuracy                           0.71     24372
   macro avg       0.57      0.70      0.55     24372
weighted avg       0.90      0.71      0.77     24372



In [6]:
# import pandas as pd
# import matplotlib.pyplot as plt
# from collections import Counter

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import (
#     classification_report,
#     average_precision_score,
# )

# from imblearn.over_sampling import SMOTE, RandomOverSampler
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.combine import SMOTETomek
# from imblearn.ensemble import BalancedRandomForestClassifier

# #  Columns to scale
# columns_to_scale = [
#     'Client_Income', 'Credit_Amount', 'Loan_Annuity',
#     'Population_Region_Relative', 'Age_Days', 'Employed_Days',
#     'Registration_Days', 'ID_Days', 'Score_Source_2',
#     'Score_Source_3', 'Phone_Change', 'Credit_Bureau',
#     'Client_Family_Members'
# ]

# #  Step 1: Split and Scale
# def stratified_split_and_scale(df, target_col='Default', test_size=0.2, random_state=42):
#     X = df.drop(columns=[target_col])
#     y = df[target_col]

#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, stratify=y, test_size=test_size, random_state=random_state
#     )

#     # For unscaled models (like Random Forest)
#     X_train_unscaled = X_train.copy()
#     X_test_unscaled = X_test.copy()

#     # Apply scaling only to selected columns
#     scaler = StandardScaler()
#     X_train_scaled = X_train.copy()
#     X_test_scaled = X_test.copy()

#     X_train_scaled[columns_to_scale] = scaler.fit_transform(X_train[columns_to_scale])
#     X_test_scaled[columns_to_scale] = scaler.transform(X_test[columns_to_scale])

#     return X_train_scaled, X_test_scaled, X_train_unscaled, X_test_unscaled, y_train, y_test, scaler

# #  Step 2: Resampling
# def apply_resampling(X, y, method='none'):
#     print(f"🔁 {method.upper()} - Before: {Counter(y)}")
    
#     if method == 'undersample':
#         sampler = RandomUnderSampler(random_state=42)
#     elif method == 'oversample':
#         sampler = RandomOverSampler(random_state=42)
#     elif method == 'smote':
#         sampler = SMOTE(random_state=42)
#     elif method == 'smote_tomek':
#         sampler = SMOTETomek(random_state=42)
#     else:
#         return X, y  # No resampling

#     X_res, y_res = sampler.fit_resample(X, y)
#     print(f" {method.upper()} - After: {Counter(y_res)}")
#     return X_res, y_res

# #  Step 3: Evaluation & Metric Collection
# def evaluate_and_collect(model, X_test, y_test, model_name, sampling_method, results):
#     y_pred = model.predict(X_test)
#     y_proba = model.predict_proba(X_test)[:, 1]

#     report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
#     print(report)
#     pr_auc = average_precision_score(y_test, y_proba)

#     results.append({
#         'Model': model_name,
#         'Sampling': sampling_method,
#         'Precision (1)': report['1.0']['precision'],
#         'Recall (1)': report['1.0']['recall'],
#         'F1-score (1)': report['1.0']['f1-score'],
#         'PR AUC': pr_auc
#     })

# # ✅ Load your data
# # user_data = pd.read_csv("loan_data.csv")
# # target_col = 'Default'

# # Replace `user_data` with your actual DataFrame
# X_train_scaled, X_test_scaled, X_train_unscaled, X_test_unscaled, y_train, y_test, scaler = stratified_split_and_scale(
#     user_data, target_col='Default'
# )

# # ✅ Define sampling methods and result holder
# resample_methods = ['none', 'undersample', 'oversample', 'smote', 'smote_tomek']
# results = []

# # ✅ Main Evaluation Loop
# for method in resample_methods:
#     print(f"\n============================")
#     print(f"🔄 Sampling: {method.upper()}")
#     print("============================")

#     # Logistic Regression (on scaled data)
#     X_res_lr, y_res_lr = apply_resampling(X_train_scaled, y_train, method)
#     lr_model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
#     lr_model.fit(X_res_lr, y_res_lr)
#     evaluate_and_collect(lr_model, X_test_scaled, y_test, 'LogisticRegression', method, results)

#     # Balanced Random Forest (on unscaled data)
#     X_res_rf, y_res_rf = apply_resampling(X_train_unscaled, y_train, method)
#     brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=42)
#     brf_model.fit(X_res_rf, y_res_rf)
#     evaluate_and_collect(brf_model, X_test_unscaled, y_test, 'BalancedRandomForest', method, results)

# # ✅ Show Final Comparison Table
# results_df = pd.DataFrame(results)
# print("\n📊 Final Comparison (Minority Class Metrics):")
# print(results_df.sort_values(by='F1-score (1)', ascending=False).round(3))

# # ✅ Optional: Export to CSV
# # results_df.to_csv("model_comparison_results.csv", index=False)


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, average_precision_score

from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from imblearn.ensemble import BalancedRandomForestClassifier

# Columns to scale
columns_to_scale = [
    'Client_Income', 'Credit_Amount', 'Loan_Annuity',
    'Population_Region_Relative', 'Age_Days', 'Employed_Days',
    'Registration_Days', 'ID_Days', 'Score_Source_2',
    'Score_Source_3', 'Phone_Change', 'Credit_Bureau',
    'Client_Family_Members'
]

# Step 1: Split and Scale
def stratified_split_and_scale(df, target_col='Default', test_size=0.2, random_state=42):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=test_size, random_state=random_state
    )

    X_train_unscaled = X_train.copy()
    X_test_unscaled = X_test.copy()

    scaler = StandardScaler()
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()
    X_train_scaled[columns_to_scale] = scaler.fit_transform(X_train[columns_to_scale])
    X_test_scaled[columns_to_scale] = scaler.transform(X_test[columns_to_scale])

    return X_train_scaled, X_test_scaled, X_train_unscaled, X_test_unscaled, y_train, y_test, scaler

# Step 2: Resampling
def apply_resampling(X, y, method='none'):
    print(f"\n🔁 {method.upper()} - Before: {Counter(y)}")

    if method == 'undersample':
        sampler = RandomUnderSampler(random_state=42)
    elif method == 'oversample':
        sampler = RandomOverSampler(random_state=42)
    elif method == 'smote':
        sampler = SMOTE(random_state=42)
    elif method == 'smote_tomek':
        sampler = SMOTETomek(random_state=42)
    else:
        return X, y

    X_res, y_res = sampler.fit_resample(X, y)
    print(f"✅ {method.upper()} - After: {Counter(y_res)}")
    return X_res, y_res

# Step 3: Evaluation with training time
def evaluate_and_collect(model, X_test, y_test, model_name, sampling_method, results, train_time):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else [0] * len(y_pred)

    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    if '1' in report:
        pr_auc = average_precision_score(y_test, y_proba)
        results.append({
            'Model': model_name,
            'Sampling': sampling_method,
            'Precision (1)': report['1']['precision'],
            'Recall (1)': report['1']['recall'],
            'F1-score (1)': report['1']['f1-score'],
            'PR AUC': pr_auc,
            'Train Time (s)': round(train_time, 2)
        })

# Load your data here
# user_data = pd.read_csv("loan_data.csv")
# Ensure 'Default' is int (0/1)
# user_data['Default'] = user_data['Default'].astype(int)

X_train_scaled, X_test_scaled, X_train_unscaled, X_test_unscaled, y_train, y_test, scaler = stratified_split_and_scale(
    user_data, target_col='Default'
)

resample_methods = ['none', 'undersample', 'oversample', 'smote', 'smote_tomek']
# resample_methods = ['none']
results = []

for method in resample_methods:
    print(f"\n============================")
    print(f"🔄 Sampling: {method.upper()}")
    print("============================")

    # Logistic Regression (on scaled)
    X_res_lr, y_res_lr = apply_resampling(X_train_scaled, y_train, method)
    lr_model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
    print("Model training started for lr_model")
    start = time.time()
    lr_model.fit(X_res_lr, y_res_lr)
    end = time.time()
    print("Model training finished for lr_model")
    evaluate_and_collect(lr_model, X_test_scaled, y_test, 'LogisticRegression', method, results, end - start)

    # Decision Tree (scaled)
    X_res_dt, y_res_dt = apply_resampling(X_train_scaled, y_train, method)
    dt_model = DecisionTreeClassifier(class_weight='balanced', max_depth=5, random_state=42)
    print("Model training started for dt_model")
    start = time.time()
    dt_model.fit(X_res_dt, y_res_dt)
    end = time.time()
    print("Model training finished for dt_model")
    evaluate_and_collect(dt_model, X_test_scaled, y_test, 'DecisionTree', method, results, end - start)

    # Random Forest (unscaled)
    X_res_rf, y_res_rf = apply_resampling(X_train_unscaled, y_train, method)
    rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
    print("Model training started for rf_model")
    start = time.time()
    rf_model.fit(X_res_rf, y_res_rf)
    end = time.time()
    print("Model training finished for rf_model")
    evaluate_and_collect(rf_model, X_test_unscaled, y_test, 'RandomForest', method, results, end - start)

    # Balanced Random Forest (no external resampling)
    if method == 'none':
        brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=42)
        print("Model training started for brf_model")
        start = time.time()
        brf_model.fit(X_train_unscaled, y_train)
        end = time.time()
        print("Model training finished for brf_model")
        evaluate_and_collect(brf_model, X_test_unscaled, y_test, 'BalancedRandomForest', method, results, end - start)

# Final results
results_df = pd.DataFrame(results)
print("\n📊 Final Comparison (Minority Class Metrics + Training Time):")
print(results_df)



🔄 Sampling: NONE

🔁 NONE - Before: Counter({0: 89608, 1: 7876})
Model training started for lr_model
Model training finished for lr_model

🔁 NONE - Before: Counter({0: 89608, 1: 7876})
Model training started for dt_model
Model training finished for dt_model

🔁 NONE - Before: Counter({0: 89608, 1: 7876})
Model training started for rf_model
Model training finished for rf_model
Model training started for brf_model
Model training finished for brf_model

🔄 Sampling: UNDERSAMPLE

🔁 UNDERSAMPLE - Before: Counter({0: 89608, 1: 7876})
✅ UNDERSAMPLE - After: Counter({0: 7876, 1: 7876})
Model training started for lr_model
Model training finished for lr_model

🔁 UNDERSAMPLE - Before: Counter({0: 89608, 1: 7876})
✅ UNDERSAMPLE - After: Counter({0: 7876, 1: 7876})
Model training started for dt_model
Model training finished for dt_model

🔁 UNDERSAMPLE - Before: Counter({0: 89608, 1: 7876})
✅ UNDERSAMPLE - After: Counter({0: 7876, 1: 7876})
Model training started for rf_model
Model training finished f

In [9]:
results_df.sort_values(by=['Model','Sampling'])

,Model,Sampling,Precision (1),Recall (1),F1-score (1),PR AUC,Train Time (s)
3,BalancedRandomForest,none,0.31,0.62,0.42,0.49,5.15
1,DecisionTree,none,0.22,0.66,0.33,0.44,0.65
8,DecisionTree,oversample,0.20,0.67,0.31,0.43,0.95
11,DecisionTree,smote,0.20,0.51,0.28,0.17,1.84
14,DecisionTree,smote_tomek,0.20,0.51,0.28,0.17,1.80
5,DecisionTree,undersample,0.21,0.66,0.32,0.42,0.09
0,LogisticRegression,none,0.00,0.00,0.00,0.08,0.12
7,LogisticRegression,oversample,0.14,0.63,0.23,0.18,1.19
10,LogisticRegression,smote,0.16,0.66,0.26,0.20,2.01
13,LogisticRegression,smote_tomek,0.14,0.63,0.23,0.18,1.19


In [10]:
import mlflow

# (Optional) Set experiment name (creates one if doesn't exist)
mlflow.set_experiment("My Loan Default Project")

with mlflow.start_run(run_name="RandomForest"):
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_metric("accuracy", 0.91)


2025/06/21 18:07:42 INFO mlflow.tracking.fluent: Experiment with name 'My Loan Default Project' does not exist. Creating a new experiment.
